## Demo: Stage2 processing for mooring data

- Apply clock offsets
- Trim to deployment period

In [1]:
import os

import yaml
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from datetime import datetime
import pandas as pd
from ctd_tools.writers import NetCdfWriter
from oceanarray import writers

moorlist = ['ds2_X_2012','ds2_X_2017','ds2_X_2018',
            'ds8_1_2012','ds9_1_2012','ds10_1_2012', 'ds11_1_2012','ds12_1_2012',
            'ds13_1_2012','ds14_1_2012','ds15_1_2012','ds16_1_2012','ds17_1_2012',
            'ds19_1_2012','ds18_1_2012','ds28_1_2017',
            'dsA_1_2018','dsB_1_2018','dsC_1_2018', 'dsD_1_2018','dsE_1_2018','dsF_1_2018',
            'dsM1_1_2017','dsM2_1_2017','dsM3_1_2017','dsM4_1_2017','dsM5_1_2017']
moorlist = ['dsE_1_2018']

In [2]:
from oceanarray.stage2 import Stage2Processor, process_multiple_moorings_stage2

# Simple usage
moorlist = ['dsE_1_2018']
basedir = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/'
results = process_multiple_moorings_stage2(moorlist, basedir)


Processing Stage 2 for mooring dsE_1_2018
Starting Stage 2 processing for mooring: dsE_1_2018
Deployment time: 2018-08-12T22:44:00.000000000
Recovery time: 2018-08-26T10:38:00.000000000
Processing sbe56 serial 6363
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 128476 to 111829 records
Final time range: 2018-08-13T12:00:00.000000000 to 2018-08-26T10:37:59.001600000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6363_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6363_use.nc
Processing sbe16 serial 2419
Removing variable: timeS
Applying clock offset: 259000 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 51670 to 47284 records
Final time range: 2018-0

/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/github/oceanarray/oceanarray/stage2.py:166: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  with xr.open_dataset(raw_filepath) as ds:
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimensi

Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 136972 to 116604 records
Final time range: 2018-08-12T22:44:08.979200000 to 2018-08-26T10:37:59.014400000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_8482_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_8482_use.nc
Processing sbe56 serial 6365
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 128443 to 111829 records
Final time range: 2018-08-13T12:00:00.000000000 to 2018-08-26T10:37:59.001600000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6365_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6365_use.nc
Processing sbe56 serial 

/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in

Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6397_use.nc
Processing sbe56 serial 6366
Applying clock offset: -2400 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 128392 to 112069 records
Final time range: 2018-08-13T11:20:00.000000000 to 2018-08-26T10:37:59.020800000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6366_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/sbe56/dsE_1_2018_6366_use.nc
Processing sbe56 serial 6394
Applying clock offset: -2400 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 128487 to 112069 records
Final time range: 2018-08-13T11:20:00.000000000 to 2018-08-26T10:37:5

...

/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in

Trimmed from 88987 to 88987 records
Final time range: 2018-08-13T11:57:36.000000000 to 2018-08-23T19:08:36.000000000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/tr1050/dsE_1_2018_13874_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/tr1050/dsE_1_2018_13874_use.nc
Processing rbrsolo serial 101645
Applying clock offset: 85860 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 131002 to 111883 records
Final time range: 2018-08-13T11:51:00.000000000 to 2018-08-26T10:38:00.000000000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/rbrsolo/dsE_1_2018_101645_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/rbrsolo/dsE_1_2018_101645_use.nc
Processing tr1050 serial 15574
App

/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in

Applying clock offset: 84420 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 130190 to 112027 records
Final time range: 2018-08-13T11:27:00.000000000 to 2018-08-26T10:38:00.000000000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/tr1050/dsE_1_2018_15577_use.nc
Successfully wrote: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/tr1050/dsE_1_2018_15577_use.nc
Processing microcat serial 7518
Applying clock offset: -2050 seconds
Trimming start to deployment time: 2018-08-12T22:44:00.000000000
Trimming end to recovery time: 2018-08-26T10:38:00.000000000
Trimmed from 124619 to 113473 records
Final time range: 2018-08-13T07:25:51.008000000 to 2018-08-26T10:37:50.979200000
Removed existing file: /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/microcat/dsE_1_2018_7518_use.nc
Succ

/Users/eddifying/Cloudfree/gitlab-cloudfree/ctd-tools/ctd_tools/writers/netcdf_writer.py:97: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  chunks.append(max(1, min(ds.dims[d], int(chunk_time))))
